<a href="https://colab.research.google.com/github/Cebedeo/03MAIR---Algoritmos-de-optimizacion---2019-Enrique-Navarro-/blob/master/Enrique_Navarro_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Actividad guiada 3 de Enrique Navarro**<br>
Enlace: https://colab.research.google.com/drive/1PxO9YIMDuNx3V9LFX5OU8toElvHv0usl


In [2]:
import urllib.request
file = "swiss42.tsp"
urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file)

!pip install tsplib95

In [0]:
import tsplib95
import random
from math import e

problem = tsplib95.load_problem(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())

#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos): 
  solucion = [0]
  for i in range(len(Nodos)-1):
    solucion = solucion + [random.choice(list(set(Nodos) - set({0}) - set(solucion)))]
  return solucion  

#crear_solucion(Nodos)

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.wfunc(a,b)

#distancia(0,1,problem)

#Devuelve la distancia total de una trayectoria
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

#solucion = crear_solucion(Nodos)
#distancia_total(solucion, problem)

#Busqueda aleatorio
def busqueda_aleatoria(problem, N):
  mejor_solucion = crear_solucion(Nodos)
  mejor_valor    = distancia_total(mejor_solucion, problem)

  for i in range(N):
      solucion = crear_solucion(Nodos)
      valor    = distancia_total(solucion, problem)
      if valor < mejor_valor:
        mejor_solucion = solucion
        mejor_valor = valor

  print("La mejor solucion es ",mejor_valor,  mejor_solucion)
   
#busqueda_aleatoria(problem, 20000)


def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):
    for j in range(i+1, len(solucion)):
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:] 
      distancia_vecina = distancia_total(vecina, problem)
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  #print(mejor_distancia)
  return mejor_solucion
      
solucion = crear_solucion(Nodos)
mejor_solucion = genera_vecina(solucion)


In [4]:
#####################################################################
def busqueda_local(problem, N):
  #problem = datos del problema
  #N = numero de iteraciones
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  solucion_referencia = crear_solucion(Nodos)
  
  for i in range(N):
    vecina = genera_vecina(solucion_referencia)
    distancia_vecina = distancia_total(vecina, problem)
    if distancia_vecina <= mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
        
    solucion_referencia = vecina
  
  print("La mejor solución encontrada es " , mejor_distancia , mejor_solucion)

busqueda_local(problem,200)

La mejor solución encontrada es  1979 [0, 34, 33, 20, 35, 36, 31, 32, 30, 28, 2, 27, 3, 4, 6, 5, 19, 13, 26, 18, 12, 11, 15, 37, 17, 7, 1, 29, 21, 24, 40, 9, 8, 14, 16, 10, 25, 41, 23, 39, 22, 38]


In [5]:
def genera_vecina_aleatorio(solucion):
  #Generador de 1 solucion vecina 2-opt (intercambiar 2 nodos)
  #Se puede mejorar haciendo que la elección no se uniforme sino entre las que estén más proximas
  i = random.choice(range(1, len(solucion)) )
  j = random.choice(list(set(range(1, len(solucion))) - {i}))
  if j<i:
    i,j=j,i
  vecina = list(solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:])
  return vecina

solucion = crear_solucion(Nodos)
print(solucion,"\n",genera_vecina_aleatorio(solucion))

[0, 21, 25, 41, 28, 1, 27, 4, 30, 22, 12, 37, 16, 6, 17, 7, 15, 10, 32, 2, 35, 34, 9, 19, 13, 38, 29, 23, 14, 20, 33, 18, 3, 5, 40, 11, 24, 26, 31, 39, 8, 36] 
 [0, 21, 25, 41, 28, 1, 27, 4, 30, 14, 12, 37, 16, 6, 17, 7, 15, 10, 32, 2, 35, 34, 9, 19, 13, 38, 29, 23, 22, 20, 33, 18, 3, 5, 40, 11, 24, 26, 31, 39, 8, 36]


In [6]:
import math

  #Funcion de probabilidad para determinar si se cambia 
#    a una solución peor respecto a la de referencia(exponencial)
def probabilidad(T,d):
  if random.random() <= math.exp(-1*d / T)   :
    return True
  else:
    return False


def bajar_temperatura(T):
  return T*.9999

def recocido_simulado(problem, TEMPERATURA=1000):
  #problem = datos del problema

  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia,problem)

  mejor_solucion = []
  mejor_distancia = 10e100

  while TEMPERATURA > 1:
    #Genera una solución vecina(aleatoria)
    vecina = genera_vecina_aleatorio(solucion_referencia) 
    
    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)
    
    #Si es la mejor solución de todas se guarda
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
      print('\t****Encontrada mejor solucion:', mejor_distancia , mejor_solucion, '\n')
    
    #Si la nueva vecina es mejor se cambia y si es peor se cambia según 
    # una probabilidad dependiente de T y de |distancia_referencia - distancia_vecina|
    p = probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) )
    if distancia_vecina < distancia_referencia or p :
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina
    
    TEMPERATURA = bajar_temperatura(TEMPERATURA)

  print("La mejor solución encontrada es :" , mejor_distancia, mejor_solucion  )
  return mejor_solucion

sol = recocido_simulado(problem, TEMPERATURA=100000)

	****Encontrada mejor solucion: 5142 [0, 23, 35, 34, 6, 12, 25, 32, 15, 22, 19, 9, 4, 16, 8, 13, 33, 14, 1, 26, 31, 39, 29, 18, 20, 5, 40, 7, 27, 24, 21, 3, 37, 30, 17, 36, 2, 11, 38, 10, 41, 28] 

	****Encontrada mejor solucion: 5043 [0, 23, 35, 34, 6, 12, 25, 26, 15, 22, 19, 9, 4, 16, 8, 13, 33, 14, 1, 32, 31, 39, 29, 18, 20, 36, 40, 7, 27, 24, 21, 3, 37, 30, 17, 5, 2, 11, 38, 10, 41, 28] 

	****Encontrada mejor solucion: 5030 [0, 23, 35, 34, 6, 12, 25, 26, 15, 39, 19, 9, 4, 16, 8, 13, 33, 14, 1, 32, 31, 22, 29, 18, 20, 36, 40, 7, 3, 24, 21, 27, 37, 30, 17, 5, 2, 11, 38, 10, 41, 28] 

	****Encontrada mejor solucion: 4905 [0, 23, 35, 34, 6, 12, 25, 26, 4, 39, 19, 9, 15, 16, 8, 13, 33, 14, 1, 32, 31, 22, 29, 18, 20, 36, 40, 7, 3, 24, 21, 27, 37, 30, 17, 5, 2, 11, 38, 10, 41, 28] 

	****Encontrada mejor solucion: 4742 [0, 21, 35, 5, 6, 12, 22, 40, 4, 39, 38, 9, 2, 16, 29, 13, 33, 14, 1, 32, 31, 26, 8, 41, 20, 36, 25, 27, 3, 24, 23, 7, 37, 30, 17, 34, 15, 11, 19, 10, 18, 28] 

	****Encon

**Colonia de hormigas**

In [10]:
####################################
#### Desarrollo para subir nota ####
def elegir_nodo(problem, H,T, pr,pd):
  #Función de probabilidad para añadir nuevo nodo.
  #Depende del peso relativo del rastro de feromonas (pr) y de la distancia entre nodos (pd)
  Nodos = list(problem.get_nodes())
  pendientes = list(set(range(1,len(Nodos))) - set(H))
  destinos = []
  
  nodo_actual = H[-1]
  total_producto = 0
  length = len(pendientes)
  for l in range(length):  #Calculo el total de feromonas y distancias de las aristas de ese nodo
    solucion_considerada = H.copy()
    solucion_considerada.append(pendientes[l]) #La solución si voy por esa arista
    inversa_distancia = 1/distancia_total(solucion_considerada, problem)
    cantidad_feromona = T[nodo_actual][pendientes[l]]
    destinos.append([cantidad_feromona,inversa_distancia])
    total_producto += (cantidad_feromona**pr)*(inversa_distancia**pd) #Se omite el valor absoluto pues no puede ser negativo

  probabilidad_destino = random.random()
  for i in range(length):  #Y ahora seleccionamos la arista a seguir
    arista = destinos[i]
    probabilidad = ((arista[0]**pr)*(arista[1]**pd))/total_producto
    probabilidad_destino -= probabilidad
    if probabilidad_destino < 0: #Seleccionado en el sorteo ponderado por probabilidades
      return pendientes[i]
    #print("Total probabilidad de ir (arista: ",arista,") al nodo",pendientes[i],":",probabilidad)
####################################

def Add_Nodo(problem, H ,T ) :
  #Establecer una una funcion de probabilidad para 
  # añadir un nuevo nodo dependiendo de los nodos mas cercanos y de las feromonas depositadas
  Nodos = list(problem.get_nodes())
  return random.choice(   list(set(range(1,len(Nodos))) - set(H) )  )


def Incrementa_Feromona(problem, T, H):
  #Incrementar segun la calidad de la solución. Añadir una cantidad inversamente proporcional a la distancia total 
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
  return T

def Evaporar_Feromonas(T):
  #Podemos elegir diferentes funciones de evaporación dependiendo de la cantidad actual y de la suma total de feromonas depositadas,...
  #Evapora 0.3 el valor de la feromona, sin que baje de 1
  T = [[ max(T[i][j] - 0.3 , 1) for i in range(len(Nodos)) ] for j in range(len(Nodos))]
  return T

#### Se modifica ligeramente para poderlo iterar en la mejora de búsqueda de parámetros (al final)
def hormigas(problem, N, pr, pd) :
  #problem = datos del problema
  #N = Número de agentes(hormigas)
  
  #Peso del rastro hormonal
  peso_rastro = pr
  #Peso de la distancia entre nodos
  peso_distancia = pd

  #Nodos
  Nodos = list(problem.get_nodes())
  #Aristas
  Aristas = list(problem.get_edges()) 
  
  #Inicializa las aristas con una cantidad inicial de feromonas:1
  T = [[ 1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]
  
  #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
  Hormiga = [[0] for _ in range(N)]
#Recorre cada agente construyendo la solución
  for h in range(N) :
    #print("\nAgente:", h)
    #Para cada agente se construye un camino
    for i in range(len(Nodos)-1) :
      
      #Elige el siguiente nodo
      #Nuevo_Nodo = Add_Nodo(problem, Hormiga[h] ,T )
      Nuevo_Nodo = elegir_nodo(problem,Hormiga[h],T,peso_rastro,peso_distancia)

      Hormiga[h].append(Nuevo_Nodo)     
    
    #Incrementa feromonas en esa arista 
    T = Incrementa_Feromona(problem, T, Hormiga[h] )
    #print("Feromonas(1)", T)
      
    #Evapora Feromonas  
    T = Evaporar_Feromonas(T)
    #print("Feromonas(2)", T)

    #Seleccionamos el mejor agente
  mejor_solucion = []
  mejor_distancia = 10e100
  for h in range(N) :
    distancia_actual = distancia_total(Hormiga[h], problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion = Hormiga[h]
      mejor_distancia =distancia_actual
  
# Modificación para poder iterar  
  #print(mejor_solucion)
  #print(mejor_distancia)
  return mejor_distancia

# Hago una búsqueda de los mejores parámetros para ACO
mejor_resultado = [0,0,9999]
resultados = []
for i in range(0,91,5):
  for j in range(0,91,5):
    resultados_tmp = 0
    for k in range(5):
      resultados_tmp += hormigas(problem,100,i,j)
      if resultados_tmp < mejor_resultado[2]:
        mejor_resultado = [i,j,resultados_tmp]
        print("Encontrado resultado mejor:",mejor_resultado)
    resultados_tmp /= 5
    resultados.append(resultados_tmp) #Me guardo los resultados medios para posible análisis estadístico posterior, que no entra en este ejercicio

    

print("El mejor resultado de ACO ha sido",mejor_resultado[2],"y se ha obtenido con peso del rastro",mejor_resultado[0]," y peso de la distancia",mejor_resultado[1])
print(matriz_resultados)


Encontrado resultado mejor: [0, 0, 4048]
Encontrado resultado mejor: [0, 5, 3789]
Encontrado resultado mejor: [0, 10, 3160]
Encontrado resultado mejor: [0, 15, 2834]
Encontrado resultado mejor: [0, 20, 2250]
Encontrado resultado mejor: [0, 25, 1950]
Encontrado resultado mejor: [0, 35, 1878]
Encontrado resultado mejor: [0, 45, 1803]
Encontrado resultado mejor: [0, 50, 1799]
Encontrado resultado mejor: [0, 60, 1789]
Encontrado resultado mejor: [0, 65, 1716]
Encontrado resultado mejor: [0, 90, 1705]
Encontrado resultado mejor: [5, 55, 1658]
Encontrado resultado mejor: [5, 60, 1656]
Encontrado resultado mejor: [5, 85, 1612]
Encontrado resultado mejor: [10, 50, 1605]
Encontrado resultado mejor: [10, 75, 1569]
Encontrado resultado mejor: [20, 75, 1530]
El mejor resultado de ACO ha sido 1530 y se ha obtenido con peso del rastro 20  y peso de la distancia 75
[4089.6, []]
